<a href="https://colab.research.google.com/github/Masum06/TinyAgent/blob/main/TinyAgent_Colab_Noebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tiktoken

# TinyAgent

In [6]:
import os, getpass
import json
import tiktoken
from openai import OpenAI

def _set_env(self, var: str, reset=False):
    if not os.environ.get(var) or reset:
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

class TinyAgent:

    def __init__(self, model, tokenizer=None, debug=False):
        self.model = model
        self.messages = []
        self.max_tokens = 10072
        self.debug = False
        self.reasoning_effort = "high"
        self.temperature = 1
        self.add_system_message(f'Keep the total tokens under (thinking + output): {self.max_tokens}')
        self.client = OpenAI()

    def clear_messages(self):
        self.messages = list()

    def add_message(self,message_type, message):
        self.messages.append({"role": message_type, "content":message})

    def add_system_message(self, message):
        self.add_message("system", message)

    def add_user_message(self, message):
        self.add_message("user", message)

    def add_assistant_message(self, message):
        self.add_message("assistant", message)

    def set_max_tokens(self, max_tokens):
        self.max_tokens = max_tokens

    def set_debug(self, debug):
        self.debug = debug

    def set_reasoning_effort(self, reasoning_effort):
        self.reasoning_effort = reasoning_effort

    def call(self, prompt="", response_type="text", cache=True):
        messages = self.messages.copy()
        if prompt:
            messages.append({"role": "user", "content":prompt})
        if cache:
            self.add_user_message(prompt)

        if "gpt-5" in self.model:
            response = self.client.responses.create(
                model=self.model,
                input=messages,
                reasoning={"effort": self.reasoning_effort},
                text={
                    "format": {
                      "type": response_type
                    },
                    "verbosity": "low"
                  },
            )
            reply = response.output_text

        elif "gpt-4" in self.model or "o3" in self.model or "o4" in self.model:
            response = self.client.responses.create(
              model=self.model,
              input=messages,
              temperature=self.temperature,
              max_output_tokens=self.max_tokens,
              top_p=1,
              text={
                "format": {
                  "type": response_type # "text", "json_object"
                }
              }
            )
            reply = response.output_text
        if self.debug:
            print(reply)
        if cache:
            self.add_assistant_message(reply)
        return reply

    def load_json(self,s):
        import json
        try:
            return json.loads(s)
        except json.JSONDecodeError:
            return None

    def call_json(self, prompt=""):
        self.add_system_message("Reply must be JSON format.")
        reply = self.call(prompt=prompt, response_type="json_object")
        if not reply:
            print("Empty reply")
            return None

        reply = reply.strip()
        if reply.startswith("```json"):
            reply = reply[len("```json"):].strip()
            if reply.endswith("```"):
                reply = reply[:-3].strip()

        # Use OR, and guard length
        if not (reply.startswith("{") and reply.endswith("}")):
            print("Not JSON structure")
            return None

        try:
            return self.load_json(reply)
        except Exception:
            print("Error parsing JSON")
            return None

In [7]:
tiny_agent = TinyAgent("o3-mini")

In [8]:
tiny_agent.messages

[{'role': 'system',
  'content': 'Keep the total tokens under (thinking + output): 10072'}]

## Unstructured Call

In [9]:
tiny_agent.call("What's your name?", cache=False)

"I'm ChatGPT, an AI language model created by OpenAI. How can I assist you today?"

In [10]:
tiny_agent.messages

[{'role': 'system',
  'content': 'Keep the total tokens under (thinking + output): 10072'}]

In [ ]:
tiny_agent.call("What can you do?", cache=True)

'I’m ChatGPT, an AI language model based on OpenAI’s GPT-4. I can help you with a wide range of tasks! Here are some things I can do:\n\n### **Text-based Assistance**\n- **Answer Questions:** Explain concepts, facts, history, and much more.\n- **Writing Help:** Draft emails, essays, articles, stories, speeches, and poems.\n- **Editing & Proofreading:** Improve clarity, grammar, and style in your writing.\n- **Summarize Content:** Summarize articles, books, or documents.\n- **Translation:** Translate between multiple languages.\n- **Research:** Gather and structure information on many topics.\n- **Idea Generation:** Brainstorm about projects, names, business ideas, or creative writing prompts.\n\n### **Technical Help**\n- **Code Writing & Debugging:** Write, explain, and debug code in many programming languages.\n- **Math Assistance:** Solve math problems, explain solutions, and guide through concepts.\n- **Data Analysis:** Help with statistics, data interpretation, and even some spread

In [ ]:
tiny_agent.messages

[{'role': 'user', 'content': 'What can you do?'},
 {'role': 'assistant',
  'content': 'I’m ChatGPT, an AI language model based on OpenAI’s GPT-4. I can help you with a wide range of tasks! Here are some things I can do:\n\n### **Text-based Assistance**\n- **Answer Questions:** Explain concepts, facts, history, and much more.\n- **Writing Help:** Draft emails, essays, articles, stories, speeches, and poems.\n- **Editing & Proofreading:** Improve clarity, grammar, and style in your writing.\n- **Summarize Content:** Summarize articles, books, or documents.\n- **Translation:** Translate between multiple languages.\n- **Research:** Gather and structure information on many topics.\n- **Idea Generation:** Brainstorm about projects, names, business ideas, or creative writing prompts.\n\n### **Technical Help**\n- **Code Writing & Debugging:** Write, explain, and debug code in many programming languages.\n- **Math Assistance:** Solve math problems, explain solutions, and guide through concepts.

## Structured Call

In [ ]:
result = tiny_agent.call_json("Give me a list of popular math equations in JSON format. Key: popular_math_equations")
result

{'popular_math_equations': [{'name': 'Pythagorean Theorem',
   'equation': 'a^2 + b^2 = c^2'},
  {'name': 'Quadratic Formula',
   'equation': 'x = [-b ± sqrt(b^2 - 4ac)] / (2a)'},
  {'name': "Euler's Formula", 'equation': 'e^{ix} = cos(x) + i*sin(x)'},
  {'name': 'Area of a Circle', 'equation': 'A = πr^2'},
  {'name': 'Slope Formula', 'equation': 'm = (y_2 - y_1) / (x_2 - x_1)'},
  {'name': 'Einstein’s Mass-Energy Equivalence', 'equation': 'E = mc^2'},
  {'name': 'Derivative Definition',
   'equation': "f'(x) = lim_{h\\to0} (f(x+h) - f(x)) / h"},
  {'name': 'Sum of a Geometric Series',
   'equation': 'S_n = a_1(1 - r^n) / (1 - r)'},
  {'name': 'Binomial Theorem',
   'equation': '(a + b)^n = \\sum_{k=0}^{n} C(n, k)a^{n-k}b^k'},
  {'name': 'Law of Sines', 'equation': 'sin(A)/a = sin(B)/b = sin(C)/c'}]}

In [ ]:
result['popular_math_equations'][0]

{'name': 'Pythagorean Theorem', 'equation': 'a^2 + b^2 = c^2'}